## Fabric Gateway Client
This notebook interacts with our Hyperledger Fabric blockchain, using our JavaScript module, as to simulate a Jupyter extension (i.e. the frontend).
Note that the code is executed on the backend (using a Jupyter kernel), whereas a Jupyter extension would run this code on the frontend (i.e. the web browser). This has two major consequences: 1. The traffic between this client and Fabric will occur via the backend (i.e. IP routing, DNS, TLS, etc. will take place within that context). 2. The user's private key is generated and stored on the backend (rather than never leaving the web browser's memory). As long as JupyterLab is executed on the *same* system (e.g. your laptop) this does not matter (both the backend and frontend are running on the same host), but be aware when tunneling JupyterLab.

### Prerequisites
We use 'IJavascript' to run JavaScript from JupyterLab (as a kernel / on the JupyterLab backend. We also install the node version specified by Fabric's documentation (v14).

In [1]:
!mamba install --yes make cxx-compiler nodejs=14.18.3


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.22.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['make', 'cxx-compiler', 'nodejs=14.18.3']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━━━━━━╸━━━━

In [2]:
!npm install -g ijavascript
!ijsinstall

npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
/opt/conda/bin/ijs -> /opt/conda/lib/node_modules/ijavascript/bin/ijavascript.js.3.0 is installed into aKmK
/opt/conda/bin/ijsinstall -> /opt/conda/lib/node_modules/ijavascript/bin/ijsinstall.js
/opt/conda/bin/ijsconsole -> /opt/conda/lib/node_modules/ijavascript/bin/ijsconsole.js
/opt/conda/bin/ijskernel -> /opt/conda/lib/node_modules/ijavascript/lib/kernel.js
/opt/conda/bin/ijsnotebook -> /opt/conda/lib/node_modules/ijavascript/bin/ijsnotebook.js

> zeromq@5.2.8 install /opt/conda/lib/node_modules/ijavascript/node_modules/zeromq
> node-gyp-build || npm run build:libzmq

+ ijavascript@5.2.1
added 8 packages from 31 contributors in 6.903s


#### Admin tasks
The organization's (Fabric and IPFS) administrator has to register a user and provision the IPFS network configuration.

**User registration**  
The Fabric CA admin registers a user (on the Docker host):

**Network provisioning**  
The organization's Fabric admin upload the IPFS private network configuration to the Fabric blockchain (on the Docker host).

### Node.js module setup

In [3]:
!npm install --only=prod ./*.tgz

npm WARN deprecated querystring@0.2.0: The querystring API is considered Legacy. new code should use the URLSearchParams API instead.
        ......] \ build:yargs: sill linkStuff yargs@16.2.0 has /home/jovyam0m
> pkcs11js@1.3.0 install /home/jovyan/work/notebook/node_modules/pkcs11js
> node-gyp rebuild

make: Entering directory '/home/jovyan/work/notebook/node_modules/pkcs11js/build'
  CXX(target) Release/obj.target/pkcs11/src/main.o
In file included from ../../nan/nan.h:58,
                 from ../src/main.cpp:1:
/home/jovyan/.cache/node-gyp/14.18.3/include/node/node.h:787:7: warning: cast between incompatible function types from 'void (*)(Nan::ADDON_REGISTER_FUNCTION_ARGS_TYPE)' {aka 'void (*)(v8::Local<v8::Object>)'} to 'node::addon_register_func' {aka 'void (*)(v8::Local<v8::Object>, v8::Local<v8::Value>, void*)'} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcast-function-type-Wcast-function-type]8;;]
  787 |       (node::addon_register_func) (regfunc),

#### Switch to JavaScript/Node.js Jupyter kernel and import the module

In [1]:
const fabric = require('jc-fabricgw-client');

### Configuration
We use a configuration file to configure our connection to the Fabric blockchain (using the Fabric Gateway service that was added in Fabric version 2.4). 

In [2]:
const CAregisterSecret = 'jwSfmSTyalJE';
let config;
fabric.getConfig('./fabric-client-config.yaml').then((result) => {config = result;}); // promises/async: https://github.com/n-riesco/ijavascript/issues/268

Configuration File: ./fabric-client-config.yaml


Promise { <pending> }

In [3]:
console.log(config);

{
  organization: 'orgA',
  mspId: 'MSPorgA',
  identity: 'janssen',
  idCertFile: './id/janssen.crt',
  idKeyFile: './id/janssen.key',
  caEndpoint: 'https://ca.orga.fabric.localhost:7054',
  caTlsCertFile: '../crypto-config/fabric/peerOrganizations/orga.fabric.localhost/ca/ca.orga.fabric.localhost-cert.pem',
  caName: 'ca.orga.fabric.localhost',
  gatewayEndpoint: 'peer0.orga.fabric.localhost:7051',
  gatewayTlsCertFile: '../crypto-config/fabric/peerOrganizations/orga.fabric.localhost/peers/peer0.orga.fabric.localhost/tls/ca.crt',
  gatewayHostAlias: 'peer0.orga.fabric.localhost'
}


#### User enrollment

In [20]:
fabric.execEnroll(config, CAregisterSecret);

 Enrolling... 


Promise { <pending> }

Enrollment complete!


#### Configure IPFS
Get the configuration for the local IPFS node from our Fabric blockchain. We will use the IPFS client Python module in the IPFS notebook to interact with IPFS.

In [4]:
fabric.execTransaction(config);

 Gateway exec... 


Promise { <pending> }

Fabric Gateway error : GatewayError: 2 UNKNOWN: evaluate call to endorser returned error: chaincode response 500, The IPFS network description with ID 'net1' does not exist
